In [6]:
from pathlib import Path
import zipfile

path_to_zip_file = "../data/Branch_user_logs.zip"
directory_to_extract_to = "../data/"

my_file = Path(path_to_zip_file)
my_dir = Path(directory_to_extract_to)
if my_file.is_file():
    print("File exists, let's unzip the file then.")
if my_dir.is_dir():
    print("Directory exists")

File exists, let's unzip the file then.
Directory exists


In [2]:
# zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
# zip_ref.extractall(directory_to_extract_to)
# zip_ref.close()

In [28]:
import os

origin_root = "../data/user_logs/"

call_list = list()
contact_list = list()
sms_list = list()

for root, dirs, files in os.walk(origin_root):
    for file in files:
        if file.endswith("collated_call_log.txt"):
            user_and_device = str(root[18:])
            user_id, device_id = user_and_device.split("/")[0], user_and_device.split("/")[1]
            tmp_txt = os.path.join(root, file)
            temp_df = pd.read_json(tmp_txt)
            temp_df['user_id'] = user_id
            temp_df['device_id'] = device_id
            call_list.append(temp_df)
        elif file.endswith("collated_contact_list.txt"):
            user_and_device = str(root[18:])
            user_id, device_id = user_and_device.split("/")[0], user_and_device.split("/")[1]
            tmp_txt = os.path.join(root, file)
            temp_df = pd.read_json(tmp_txt)
            temp_df['user_id'] = user_id
            temp_df['device_id'] = device_id
            contact_list.append(temp_df)
        elif file.endswith("collated_sms_log.txt"):
            user_and_device = str(root[18:])
            user_id, device_id = user_and_device.split("/")[0], user_and_device.split("/")[1]
            tmp_txt = os.path.join(root, file)
            temp_df = pd.read_json(tmp_txt)
            temp_df['user_id'] = user_id
            temp_df['device_id'] = device_id
            sms_list.append(temp_df)
        

call_df = pd.concat(call_list, axis=0)
contact_df = pd.concat(contact_list, axis=0)
sms_df = pd.concat(sms_list, axis=0)

In [39]:
sms_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1334464 entries, 0.0 to 3807.0
Data columns (total 9 columns):
contact_id      1334464 non-null float64
datetime        1334464 non-null object
device_id       1334464 non-null object
item_id         1334464 non-null float64
message_body    1334464 non-null object
sms_address     1333671 non-null object
sms_type        1334464 non-null float64
thread_id       1334464 non-null float64
user_id         1334464 non-null object
dtypes: float64(4), object(5)
memory usage: 101.8+ MB


In [44]:
sms_df.sample(10)

,contact_id,datetime,device_id,item_id,message_body,sms_address,sms_type,thread_id,user_id
5304.0,0.0,2017-02-16 06:33:02.026000,device-1,7889.0,They are stopped from evicting you until 20/3/...,+254524492681,2.0,701.0,user-285
2305.0,0.0,2017-02-28 16:30:54.575000,device-1,377.0,You attempted to call me but I was not availab...,0710395297,1.0,218.0,user-220
6080.0,0.0,2017-02-05 14:08:24.673000,device-1,15746.0,Usishke ata,+254977196186,1.0,192.0,user-35
18117.0,0.0,2017-02-06 11:12:10.701000,device-1,11160.0,Nko subaru,+254362000898,2.0,222.0,user-69
564.0,0.0,2017-09-22 09:13:33.688000,device-1,3622.0,Your bundle balance : 70MB. Expiry: 23-09-17.M...,AIRTEL,1.0,1119.0,user-124
4417.0,0.0,2017-10-04 20:17:25.203000,device-2,48499.0,"My phone might be stolen,Please keep the numbe...",0724849506,5.0,1598.0,user-165
3293.0,0.0,2017-05-30 17:27:56.032000,device-1,1198.0,Anyway thankyou too for been more than a Frd ...,+254083068493,2.0,6.0,user-173
2289.0,0.0,2017-03-12 16:02:41.027000,device-1,1670.0,NDIO tunaosha maviombo,0728136939,2.0,174.0,user-224
103.0,0.0,2017-10-25 04:18:53.828000,device-1,539.0,Amerauka kunikol but I can't receive,+254640243112,2.0,4174.0,user-133
10076.0,0.0,2017-04-16 17:50:20.585000,device-1,2462.0,They are still discussing,+254067105656,2.0,59.0,user-261


In [46]:
sms_df.sms_address.value_counts()

Safaricom        99918
MPESA            78152
79079            16527
+254911198179    12783
SAFARICOM        11654
+254953922231     8938
+254887092383     8133
Branch-Co         7384
+254854331893     6704
+254535684613     6300
+254171590107     6208
+254710104699     5824
+254331030276     5386
+254228667815     5376
+254934030746     5335
+254913970553     5287
0708144690        5054
+254212687294     4954
+8988             4787
+254010948652     4742
+254426761486     4734
+254038290966     4562
+254171397045     4136
+254201417976     4028
+254631796039     3863
+254963951209     3771
+254073214049     3756
LOTTO             3748
+254715469790     3619
+254277654664     3579
                 ...  
0729108051           1
+254678499467        1
0755244934           1
+254552327675        1
+254212748964        1
0727746473           1
+254879612297        1
254438523222         1
+254116254661        1
+254533171371        1
0708479995           1
+254408406438        1
0780342838 

In [40]:
call_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 635290 entries, 0.0 to 5375.0
Data columns (total 13 columns):
cached_name          401594 non-null object
call_type            635290 non-null float64
country_iso          388772 non-null object
data_usage           388772 non-null float64
datetime             635290 non-null datetime64[ns]
device_id            635290 non-null object
duration             635290 non-null float64
features_video       388772 non-null object
geocoded_location    311410 non-null object
is_read              588829 non-null object
item_id              635290 non-null float64
phone_number         635290 non-null object
user_id              635290 non-null object
dtypes: datetime64[ns](1), float64(4), object(8)
memory usage: 67.9+ MB


In [57]:
call_df.country_iso.value_counts()

KE    386488
US      1968
QA       144
GB        85
BI        62
SO        22
RW         2
UG         1
Name: country_iso, dtype: int64

In [41]:
contact_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 229434 entries, 0.0 to 294.0
Data columns (total 9 columns):
date_added             355 non-null object
device_id              229434 non-null object
display_name           228712 non-null object
item_id                229434 non-null float64
last_time_contacted    229434 non-null float64
phone_numbers          217727 non-null object
photo_id               9815 non-null float64
times_contacted        229434 non-null float64
user_id                229434 non-null object
dtypes: float64(4), object(5)
memory usage: 17.5+ MB


In [47]:
contact_df.sample(10)

,date_added,device_id,display_name,item_id,last_time_contacted,phone_numbers,photo_id,times_contacted,user_id
279.0,NaN,device-1,Bever,30876.0,0.000000e+00,"[{'item_id': 68027, 'phone_number': '072301223...",NaN,0.0,user-158
9147.0,NaN,device-1,Senge,11278.0,0.000000e+00,"[{'item_id': 24265, 'normalized_phone_number':...",NaN,0.0,user-363
19.0,NaN,device-1,Mama Shamba,50890.0,1.484382e+12,"[{'item_id': 130178, 'normalized_phone_number'...",NaN,6.0,user-149
1.0,NaN,device-1,gapereju,4039.0,0.000000e+00,"[{'item_id': 8190, 'normalized_phone_number': ...",NaN,0.0,user-93
327.0,NaN,device-1,Kor Kangala,147717.0,0.000000e+00,"[{'item_id': 319640, 'normalized_phone_number'...",NaN,0.0,user-69
1063.0,NaN,device-1,Doreen Makena Cl,12762.0,1.508401e+12,"[{'item_id': 38600, 'normalized_phone_number':...",NaN,16.0,user-136
5.0,NaN,device-6,Augastine,513.0,0.000000e+00,"[{'item_id': 9655, 'normalized_phone_number': ...",NaN,0.0,user-298
608.0,NaN,device-5,Mma jem,284858.0,0.000000e+00,"[{'item_id': 578611, 'normalized_phone_number'...",NaN,0.0,user-184
1269.0,NaN,device-1,David Korir sofa,1008845.0,0.000000e+00,"[{'item_id': 7179287, 'normalized_phone_number...",NaN,0.0,user-302
286.0,NaN,device-2,Wanugi,52898.0,0.000000e+00,"[{'item_id': 134162, 'normalized_phone_number'...",NaN,0.0,user-258


In [54]:
contact_df.iloc[608, 5]

[{'item_id': 19996,
  'normalized_phone_number': '+254499986739',
  'phone_number': '0708095481'}]

In [1]:
import pandas as pd
import numpy as np
testing_txt = "../data/user_logs/user-1/device-1/collated_call_log.txt"
df = pd.read_json(testing_txt)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 0 to 181
Data columns (total 11 columns):
cached_name          126 non-null object
call_type            182 non-null int64
country_iso          182 non-null object
data_usage           182 non-null int64
datetime             182 non-null datetime64[ns]
duration             182 non-null int64
features_video       182 non-null bool
geocoded_location    180 non-null object
is_read              182 non-null bool
item_id              182 non-null int64
phone_number         182 non-null object
dtypes: bool(2), datetime64[ns](1), int64(4), object(4)
memory usage: 14.6+ KB


In [5]:
import pandas as pd
import numpy as np

pd.__version__
np.__version__

'1.14.0'

In [2]:
testing_txt = "../data/user_logs/user-309/device-1/collated_call_log.txt"
s = pd.read_json(testing_txt)